In [1]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructType,StructField,StringType,LongType,DoubleType,FloatType

import statsmodels.tsa.api as sm
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession

import warnings
warnings.filterwarnings('ignore')

In [2]:
spark = SparkSession.builder.appName('TSF-pollution').getOrCreate()

In [3]:
df = pd.read_csv("dataset/air_quality_index.csv")

In [4]:
data = spark.read.format('csv').options(header='true', inferSchema='true').load('dataset/air_quality_index.csv').select('COUNTRY','CITY','VALUE')


In [103]:
schema = StructType([StructField('COUNTRY', StringType(), True),
                     StructField('CITY', StringType(), True),
                     StructField('VALUE', DoubleType(), True)])

In [104]:
selected_com = data.groupBy(['COUNTRY','CITY']).count().filter("count > 104").select("COUNTRY","CITY")
data_selected_store_departments = data.join(selected_com, ['COUNTRY','CITY'],'inner')


In [105]:
df.set_index('DATE',inplace = True)

In [ ]:
def holt_winters_time_series_udf(data):
  
    # df.set_index('DATE',inplace = True)
    time_series_data = df['VALUE']
    
    ##the model
    model_monthly = sm.ExponentialSmoothing(np.asarray(time_series_data),trend='add').fit()

    ##forecast values
    forecast_values = pd.Series(model_monthly.forecast(1),name = 'fitted_values')
    
    return pd.DataFrame({'COUNTRY': [str(data.COUNTRY.iloc[0])],'CITY': [str(data.CITY.iloc[1])],'VALUE': [forecast_values[0]]})


In [107]:
##aggregating the forecasted results in the form of a spark dataframe
forecasted_spark_df = df.groupby(['COUNTRY','CITY']).apply(holt_winters_time_series_udf)


In [108]:
forecasted_spark_df.head()

COUNTRY        CITY      VALUE
COUNTRY CITY                                       
IN      Bengaluru  0      IN   Bengaluru  48.186877
        Bhopal     0      IN      Bhopal  48.186877
        Chandigarh 0      IN  Chandigarh  48.186877
        Chennai    0      IN     Chennai  48.186877
        Delhi      0      IN       Delhi  48.186877